In [ ]:
import numpy as np
from numpy import ndarray
import math
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as p

# Part III
### Learning

Vi finder først en måde/metode til at estimere parametrene fra noget observeret data. Dette bruger vi også senere til EM algoritmen. Vi omskriver forward_sim uden faste variabel deklerationer. 

In [ ]:
alpha=0.9 # Probability parameter alpha in (0.5, 1) for P(Z|C)
beta=0.2  # Transition probability parameter for moving to and from parallel processing
gamma=0.1  # Transition probability parameter between states
lmda=(1, 5)  # Rate parameters for the Poisson distribution modeling spike counts under each attention state.

# Transition probability matrix for the HMM, parameterized by gamma and beta (Transition probabilities of switching between states C in {0, 1, 2}).
Gamma = np.array([[1-gamma, 0, gamma], [0, 1-gamma, gamma], [beta*0.5, beta*0.5, 1-beta]])
Z_prob = [[alpha, 1-alpha], [1-alpha, alpha], [0.5, 0.5]]

def forward_sim(n, T, alpha=0.9, gamma=0.1, beta=0.2, lmda=(1, 5)): 
    
    # Initialize arrays
    C = np.zeros(T, dtype=int)
    Z = np.zeros((T, n), dtype=int)
    X = np.zeros((T, n), dtype=int)

    # Simulate the proces for the remaining t=1 to t=T timesteps
    for t in range(1, T):
        C[t] = random.choices([0, 1, 2], Gamma[2 if t == 0 else C[t-1]])[0] # First state is 0, every other picked random via the Gamma transition matrix
        for i in range(0, n):
            Z[t, i] = random.choices([0, 1], Z_prob[C[t]])[0] # Simulated hidden variables Z drawn from the distribution P(Z|C)
            X[t, i] = np.random.poisson(lam = lmda[Z[t, i]]) # Simulated observed variables drawn at random from the poisson distribution.

    return C, Z, X

Efter at have en måde til at estimere nogen parametre, kan vi antage at det er de rigtige parametre, og bruge dette til at infer nye c og z. 

Initialization

In [ ]:
### Simulate Data for T timesteps and n neurons ###
T = 100  # Time period
n = 10   # Number of neurons

# The C_0 vector which would correspond to C[-1], but is redundant for the purposes of the C array, 
# which corresponds to a C for every T. In reality we have T+1 C variables and T x N Z and X variables.
C_initial = np.array([0, 0, 1]) 
C, Z, X = forward_sim(n, T,alpha=0.9, gamma=0.1, beta=0.2, lmda=(1, 5))

In [ ]:
# Function for P(X|Z)
def PX_given_Z(x, z, lmda: tuple):
    return (np.exp(-1 * lmda[z]) * ((lmda[z]**x)/math.factorial(x)))



In [ ]:
def Z_inferred(lmda: tuple, alpha: float):
    #For every observed X=x, we calculate the conditional probaility P(Z=0|X=x)
    #We use Bayes theorem to do this. 
    Z_inferred = np.zeros((T, n, 2), dtype=float) 

    Z_marg = (1-alpha, alpha) #We assume that P(Z=0) = 0.5 due to the distribution of Z | C, and due to an assumption that P(C=1) = P(C=0)

    #We start by calculating P(X) for every observed X and save them in a table. It doesn't make sense to compute P(X) for unobserved values of X
    X_marg = np.zeros(np.max(X) + 1)
    #We calculate P(X) using the formula: P(X=x) = P(X=x|Z=0)P(Z=0) + P(X=x|Z=1)P(Z=1)
    #This works because Z ony takes values in {0,1}. We simply obtain the joint distribution P(X,Z) and sum out Z

    for i in range(len(X_marg)):
        X_marg[i] = PX_given_Z(i, 0, lmda)*Z_marg[0] + PX_given_Z(i, 1, lmda)*Z_marg[1] 
    #X_marg[i] holds the answer for the query P(X=i), Likewise Z_marg[i] holds the answer for the query p(Z=i)

    #For every observed X=x, we calculate the conditional probaility P(Z=0|X=x)
    #We use Bayes theorem to do this. 
    for i in range(Z_inferred.shape[0]): 
        for j in range(Z_inferred.shape[1]): 
            Z_inferred[i, j, 0] = (PX_given_Z(X[i, j], 0, lmda)*Z_marg[0])/(X_marg[X[i, j]])
            Z_inferred[i, j, 1] = 1 - Z_inferred[i, j, 0] 


In [ ]:
# Function for P(Z|C)
def PZ_given_C(c, z, alpha):
    Z_prob = np.array([[alpha, 1-alpha], [1-alpha, alpha], [0.5, 0.5]])
    return Z_prob[c, z]

In [ ]:
# Helper function for Forward Algorithm
def calc_emission_proba(t: int, c: int, lmda: tuple, alpha: float) -> float:
    """
    Calculates the emission probabilities at timestep t
    The probabilities given by P(X_1,t...X_n,t|C_t)
    P(Z|C) by function PZ_given_C(c, z)
    P(X|Z) Poisson distribution given by PX_given_Z(x, z)
    """
    emission_proba = 0.
    for i in range(n):
        emission_proba += np.log(PZ_given_C(c, 0, alpha)*PX_given_Z(X[t, i], 0, lmda) + PZ_given_C(c, 1, alpha)*PX_given_Z(X[t, i], 1, lmda))
    emission_proba = np.exp(emission_proba)
    return emission_proba


In [ ]:
################## Forward Algorithm ##################
def forward(alpha: float, beta: float, gamma: float, lmda: tuple) -> ndarray:
    """
    Iteratively updates the Alpha_t vectors in the Alpha matrix.
    The Alpha matrix represents the P(C|X) distribution for each timestep T as an T x 3 matrix.
    """
    # Initialization of the forward probabilities matrix, Alpha.
    Alpha = np.zeros((T, 3), dtype=float)  # T x 3 matrix P(C|X)

    # Transition probability matrix for the HMM, parameterized by gamma and beta (Transition probabilities of switching between states C in {0, 1, 2}).
    Gamma = np.array([[1-gamma, 0, gamma], [0, 1-gamma, gamma], [beta*0.5, beta*0.5, 1-beta]])

    # Buttom-up approach for updating beliefs alpha_t for forward algorithm on C.
    for t in range(T):
        for c in range(3):
            c_prev = 0. # For calculating SUM_c_t-1 P(C_t|c_t-1)*Alpha_t-1(c_t-1)
            for c_ in range(3):
                c_prev += Gamma[c_, c]*C_initial[c_] if t == 0 else Alpha[t-1, c_]
            Alpha[t, c] = calc_emission_proba(t, c, lmda, alpha)*c_prev # Alpha_t = P(X_1,t,...,X_n,t|C_t) * SUM_c_t-1 P(C_t|c_t-1)*Alpha_t-1(c_t-1)
        row_sum = np.sum(Alpha[t], axis=0) # Normalizing sum for each row
        Alpha[t] = Alpha[t]/row_sum # Row-wise normalization for the CPD, normalizing the unnormalized Gibbs measure of each row.
    return Alpha

In [ ]:
def inferred_C_Z(alpha: float, beta: float, gamma: float, lmda: tuple):
    """
    Infers the CPD of P(C|Z) as an T x 3 Alpha matrix
    And the CPD P(Z|X) as an T x n Z_infered matrix
    """
    Alpha = forward(alpha, beta, gamma, lmda)
    Z_inf = Z_inferred(lmda, alpha)
    return Alpha, Z_inf

In [ ]:
Alpha, Z_inf = inferred_C_Z(alpha, beta, gamma, lmda)
Alpha[30:50] # P(C|X) of the 19 timesteps t=30 to t=49
C[30:50] # Simulated values of the hidden layer C from t=30 to t=49

Prøver at lave en generel metode til at cementere værdierne til at være ny data.

In [ ]:
T = 100  # Time period
n = 10   # Number of neurons



In [ ]:
print(C[0])

Cementering ud fra vores inference

In [ ]:
def cement_data(inferd_Z, inferd_C):
    Z = np.zeros((T,n),dtype=int)
    C = np.zeros(T,dtype=int)

    for t in range(1, T):
        C[t] = np.argmax(inferd_C[t])
        for i in range(n):
            Z[t,i] = np.argmax(inferd_Z[t][i]) #giver det index hvor der er størst værdi i vores confidence matrix
    
    return [Z, C] #liste med 2 entries, Z og C

Parameter finde funktioner

In [ ]:

def find_param(Z_hat, C_hat):
    """ 
    find_param updates parameters from new infered Z and C
    Input: Array Z with shape [n,T], array C with shape [T,]
    Returns: alpha, beta, gamma, lambda0, lambda1
    """
    T = C_hat.shape[0]
    n = Z_hat.shape[1]
    
    _beta    = 0.
    _gamma   = 0.
    _Z = (C_hat == 1).sum() * n
    _W = (C_hat == 2).sum() 
    
    _lambda0 = X[Z_hat == 0].sum() * (1/(Z_hat == 0).sum()) #Da både X og Z har samme shape kan vi bare tælle de indexer 
    _lambda1 = X[Z_hat == 1].sum() * (1/(Z_hat == 1).sum()) 
    
    _alpha = 1/_Z * (Z_hat[C_hat == 1].sum())
    
    for t in range(T-1):
        _beta  += 1 if (C_hat[t] < C_hat[t+1]) else 0
        _gamma += 1 if (C_hat[t] > C_hat[t+1]) else 0
    
    _beta = 1/(T - _W) * _beta
    _gamma = 1/_W * _gamma
    
    return _alpha, _beta, _gamma, _lambda0, _lambda1

find_param(Z,C)